# A step-by-step tutorial of pydocker usage

In [4]:
from dsd.sys.docker.pydocker import pydocker
from io import BytesIO
import os

In [5]:
docker = pydocker()

init pydocker


In [6]:
docker

<dsd.sys.docker.pydocker.pydocker instance at 0x7f36196d0ab8>

# 1. Build
## default dockerfile name is $Dockerfile$
Dockerfile:
```
# For Test
FROM ubuntu:14.04
MAINTAINER zy <zy3381@gmail.com>
EXPOSE 9999
```
## returns: build logs

In [7]:
imageTag = 'joe/buildtest'
response = docker.build(tag=imageTag, dockerfilePath='../docker/test')
if response:
    for line in response:
        print line
else:
    print 'build failed'

# For Test
FROM ubuntu:14.04
MAINTAINER zy <zy3381@gmail.com>
EXPOSE 9999

{"stream":"Step 1 : FROM ubuntu:14.04\n"}

{"stream":" ---\u003e 06ab2de020f4\n"}

{"stream":"Step 2 : MAINTAINER zy \u003czy3381@gmail.com\u003e\n"}

{"stream":" ---\u003e Using cache\n"}

{"stream":" ---\u003e 22184b86dc3e\n"}

{"stream":"Step 3 : EXPOSE 9999\n"}

{"stream":" ---\u003e Using cache\n"}

{"stream":" ---\u003e bb8849776236\n"}

{"stream":"Successfully built bb8849776236\n"}



# 2. list images

In [8]:
pses = docker.images()
for line in pses:
    print line['RepoTags']

joe/buildtest:latest
wzz/lasagne:latest
kaixhin/cuda-theano:7.0
shawn/cuda-mxnet:422
<none>:<none>
<none>:<none>
cuda-mxnet:20160421
kaixhin/cudnn:7.0
wangjianyong/mxnet_jupyter:latest
caffe:latest
guoq/imxnet:cuda-7.0
<none>:<none>
ubuntu:14.04
hello-world:latest
kaixhin/cuda-digits:latest
kaixhin/cuda:7.0


# 3. run
## $imageTag$ comes from step 1
## params:
## name='myUbuntuTest',
## ports={8888:1234, 7777:5678},
## volumes={'/home/zhouyao/':'/mnt/homeOfJoe'}
## Returns: *container handle*
## net stat: *9999/tcp, 0.0.0.0:5678->7777/tcp, 0.0.0.0:1234->8888/tcp*
## volume in host: */home/zhouyao/*, volume in container: */mnt/homeOfJoe*

In [11]:
container = docker.run(image=imageTag, 
           detach=True, 
           stdin_open=True, 
           tty=True, 
           command='bash', 
           name='myUbuntuTest',
           user=None,
           ports={8888:1234, 7777:5678},
           volumes={'/home/zhouyao/':'/mnt/homeOfJoe'}
           )
if container is not None:
    print 'container is runing with id:'
    print container.get('Id')
else:
    print 'create container error'

container is runing with id:
cdf671ad74a9063f8a4481e487b86638d432fe52b299335a5654d6bff648240c


# 4. start
## *container_id* comes from step 3

In [12]:
container_id = container.get('Id')
response = docker.start(container=container_id)
if response:
    print 'container ', container_id, ' started'
else:
    print 'starting container error'

container  cdf671ad74a9063f8a4481e487b86638d432fe52b299335a5654d6bff648240c  started


# 5. check status

In [13]:
pses = docker.ps()
print 'Runing containers:'
print '##################'
for line in pses:
    print line['image'], line['name']

Runing containers:
##################
joe/buildtest /myUbuntuTest
wzz/lasagne /wzz_docker
guoq/imxnet:cuda-7.0 /lwb-mxnet
sha256:c832a806ee999c1091fc279fd427bf331e838c57789e21d9d59ed4b98826c377 /wjy-mxnet


# 6. Stop 

In [14]:
container_id = container.get('Id')
response = docker.stop(container=container_id,
                       timeout=0)
if response:
    print 'container', container_id, 'stop succeed'
else:
    print '#######################'
    print 'container stop failed'

container cdf671ad74a9063f8a4481e487b86638d432fe52b299335a5654d6bff648240c stop succeed


# 7. Remove container

In [15]:
container_id = container.get('Id')
response = docker.rm(container=container_id)
if response:
    print 'container ', container_id, 'removed succed'
else:
    print '#######################'
    print 'container removed failed'

container  cdf671ad74a9063f8a4481e487b86638d432fe52b299335a5654d6bff648240c removed succed


# 8. Remove image

In [16]:
imageTag = 'joe/buildtest'
response = docker.rmi(image=imageTag,
                      force=False,
                      noprune=None)
if response:
    print 'image ', imageTag, 'removed succeed'
else:
    print '#######################'
    print 'image removed failed'

image  joe/buildtest removed succeed
